In [2]:
import pandas as pd
import numpy as np

In [5]:
carbon_df = pd.read_csv('City_Carbon_Stock.csv', sep=';')
carbon_df.head()

,system:index,CA,CB,CS,latitude,longitude,.geo
0,0,50.765.053,12.045.225,6,-7.496.441.045.977.400,11.065.896.827.426.300,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
1,1,46.980.125,11.828.725,5,-7.487.457.893.136.210,1.106.050.693.572.160,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2,2,463.192,11.608.475,5,-7.487.457.893.136.210,11.061.405.251.005.700,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
3,3,35.962.925,88.882,5,-7.487.457.893.136.210,11.062.303.566.289.800,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
4,4,33.521.976,831.685,5,-7.487.457.893.136.210,11.063.201.881.573.900,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
